In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix # Confusion matrix
from sklearn.metrics import classification_report # precision, recall, f1

#from sklearn.metrics import f1_score
#from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Capstone_project/cleaned_reviews.csv')
df.head()

,cleaned_reviews,rating
0,excellent,5
1,dog become healthy thank,5
2,quality service everyone happy,5
3,went top dog raw food morning first time amazi...,5
4,very stylish site enough information make choi...,5


In [5]:
def get_index(x):
  
  index_1, index_5 = [], []

  for ind,val in enumerate(x):

    if val == 1:
      index_1.append(ind)
    elif val == 5:
      index_5.append(ind)
     
  print(len(index_1), len(index_5))
  
  index_trn = index_1[:70000] + index_5[:70000]
  index_tst = index_1[70000:] + index_5[70000:]
 
  return index_trn,index_tst

In [6]:
index_train, index_test = get_index(df['rating'])

df_train = df.iloc[index_train]
df_test = df.iloc[index_test]

df_train['rating'] = df_train['rating'].replace([1,5],[0,1])
df_test['rating'] = df_test['rating'].replace([1,5],[0,1])

97645 925883


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
x = df_train.cleaned_reviews
y = df_train.rating

In [8]:
x_validation_and_test = df_test.cleaned_reviews
y_validation_and_test = df_test.rating

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec1 = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
tvec1.fit(x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0,
                max_features=100000, min_df=1, ngram_range=(1, 3), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [10]:
SEED = 2000
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [11]:
x_train_tfidf = tvec1.transform(x)
x_validation_tfidf = tvec1.transform(x_validation)
x_test_tfidf = tvec1.transform(x_test)

In [13]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB(alpha=0.0001,fit_prior=True)
clf.fit(x_train_tfidf, y)
pred = clf.predict(x_validation_tfidf)

print(confusion_matrix(y_validation, pred))

# Precision, Recall, F1-score 
print(classification_report(y_validation, pred))

[[ 11685   2274]
 [ 68170 359635]]
              precision    recall  f1-score   support

           0       0.15      0.84      0.25     13959
           1       0.99      0.84      0.91    427805

    accuracy                           0.84    441764
   macro avg       0.57      0.84      0.58    441764
weighted avg       0.97      0.84      0.89    441764



In [9]:
clf_lr = LogisticRegression()
print(clf_lr)
clf_lr.fit(x_train_tfidf, y_train)
pred_lr = clf_lr.predict(x_validation_tfidf)

print(confusion_matrix(y_validation, pred_lr))

# Precision, Recall, F1-score 
print(classification_report(y_validation, pred_lr))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
[[10648  1669]
 [  813 11281]]
              precision    recall  f1-score   support

           0       0.93      0.86      0.90     12317
           1       0.87      0.93      0.90     12094

    accuracy                           0.90     24411
   macro avg       0.90      0.90      0.90     24411
weighted avg       0.90      0.90      0.90     24411



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
